# Comparison with pymatgen

The `pymatgen` project also has [tools capable of calculating the mean-squared displacement and diffusion coefficient](https://pymatgen.org/addons#add-ons-for-analysis) from a relevant input. 
So why should you use `kinisi` over `pymatgen`?

The simple answer is that the approach taken by `kinisi`, which is outlined in the [methodology](./methodology.html), uses a higher precision approach to estimate the diffusion coefficent and offers an accurate estimate in the variance of the mean-squared displacements and diffusion coefficient from a single simulation. 

In this notebook, we will compare the results from `pymatgen` and `kinisi`. 
First we will import the `kinisi` and `pymatgen` `DiffusionAnalyzer` classes. 

In [ ]:
import numpy as np
from kinisi.analyze import DiffusionAnalyzer as KinisiDiffusionAnalyzer
from pymatgen.analysis.diffusion.analyzer import DiffusionAnalyzer as PymatgenDiffusionAnalyzer
from pymatgen.io.vasp import Xdatcar
np.random.seed(42)

The `kinisi.DiffusionAnalyzer` API was based on the `pymatgen` equivalent, therefore, the two take the same inputs and can parse the `Xdatcar.structures`. 

In [ ]:
p_params = {'specie': 'Li',
            'time_step': 2.0,
            'step_skip': 50}

In [ ]:
xd = Xdatcar('./example_XDATCAR.gz')

We can then run both the `pymagten` analysis and the `kinisi` analysis (the `pymatgen` requires and additional `temperature` keyword which is not used in this example). 

In [ ]:
pymatgen_diff = PymatgenDiffusionAnalyzer.from_structures(
    xd.structures, temperature=300, **p_params)

In [ ]:
kinisi_diff = KinisiDiffusionAnalyzer.from_pymatgen(
    xd.structures, parser_params=p_params)

Now we can plot the mean-squared displacement from each to check agreement, the `pymatgen` time units are femtoseconds so these are adjusted. 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(pymatgen_diff.dt / 1000, pymatgen_diff.msd, label='pymatgen')
plt.plot(kinisi_diff.dt, kinisi_diff.msd, label='kinisi')
plt.legend()
plt.ylabel('MSD/Å$^2$')
plt.xlabel('$\Delta t$/ps')
plt.show()

We can see that the results overlap almost entirely.

However, this doesn't show the benefits for using `kinisi` over `pymatgen`. 
The first benefit is that `kinisi` will accurately estimate the variance in the observed mean-squared displacements, giving error bars for the above plot. 

In [ ]:
plt.errorbar(kinisi_diff.dt, kinisi_diff.msd, kinisi_diff.msd_std, c='#ff7f0e')
plt.ylabel('MSD/Å$^2$')
plt.xlabel('$\Delta t$/ps')
plt.show()

The second benefit is that `kinisi` will estimate the diffusion coefficient with an accurate uncertainty. 
`pymatgen` also estimates this uncertainty, however, `pymatgen` assumes that the data is independent and applies [weighted least squares](https://en.wikipedia.org/wiki/Weighted_least_squares). 
However, mean-squared displacement observations are inherently dependent (as discussed in the [thought experiment in the methodology](https://kinisi.readthedocs.io/en/latest/methodology.html#Understanding-the-correlation-between-measurements)), so `kinisi` accounts for this and applied a [generalised least squares](https://en.wikipedia.org/wiki/Generalized_least_squares) style approach. 
This means that the estimated variance in the diffusion coefficient from `kinisi` is accurate (while, `pymatgen` will heavily underestimate the value) and given the [BLUE](https://en.wikipedia.org/wiki/Gauss–Markov_theorem#Generalized_least_squares_estimator) nature of the GLS approach, `kinisi` has a higher probability of determining a value for the diffusion coefficient closer to the true diffusion coefficient. 

In [ ]:
kinisi_diff.diffusion(kinisi_diff.ngp_max)

In [ ]:
from uncertainties import ufloat

In [ ]:
print('D from pymatgen:', 
      ufloat(pymatgen_diff.diffusivity, pymatgen_diff.diffusivity_std_dev))

In [ ]:
print('D from kinisi:', 
      ufloat(np.mean(kinisi_diff.D), np.std(kinisi_diff.D, ddof=1)))

The comparison between weighted and generalised least squared estimators will be discussed in full in a future publication covering the methodology of `kinisi`.